In [30]:
import pyomo.environ as pyo
from random import randint
from pyomo.environ import (
    ConcreteModel,
    Set,
    Var,
    Binary,
    Objective,
    Constraint,
    SolverFactory,
    NonNegativeReals,
    Reals,
    ConstraintList,
    minimize,
    SolverStatus,
    TerminationCondition,
)

In [31]:
sites = [1, 2, 3]
clients = [1, 2, 3, 4, 5]
demand = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}
fixed_cost = {i: 1000 for i in sites}  # Coût fixe d'ouverture des sites
serve_cost = {
 (1, 1): 4,
 (2, 1): 5,
 (3, 1): 6,
 (4, 1): 8,
 (5, 1): 10,
 (1, 2): 6,
 (2, 2): 4,
 (3, 2): 3,
 (4, 2): 5,
 (5, 2): 8,
 (1, 3): 9,
 (2, 3): 7,
 (3, 3): 4,
 (4, 3): 3,
 (5, 3): 4,
}

M = {
    1: 500,
    2: 500,
    3: 500,
} # maximum amount that may be handled yearly

# Paramètres
K = 2  # Nombre maximal d'installations à ouvrir

In [98]:
# Création du modèle
model = ConcreteModel()

# Déclaration des ensembles
model.sites = Set(initialize=sites)
model.clients = Set(initialize=clients)

# Déclaration des variables
model.x = Var(model.clients, model.sites, within=NonNegativeReals)
model.y = Var(model.sites, within=Binary)

# Fonction objectif
model.obj = Objective(
    expr=sum(fixed_cost[j] * model.y[j] for j in model.sites) +
         sum(serve_cost[i, j] * model.x[i, j] for i in model.clients for j in model.sites),
    sense=minimize
)

# Contraintes
model.demand_constraint = Constraint(model.clients,
                                     rule=lambda model, i: sum(model.x[i, j] for j in model.sites) >= demand[i])
model.service_constraint = Constraint(model.sites,
                                      rule=lambda model, j: -sum([model.x[i, j] for i in model.clients]) >= - M[j] * model.y[j])
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# model.facility_constraint = Constraint(expr=sum(model.y[i] for i in model.sites) <= K)

# Résolution du problème
solver = SolverFactory('cbc')
results = solver.solve(model, tee=False)

# Affichage de la solution
print("Coût total:", model.obj())
print("Sites ouverts:", [i for i in model.sites if model.y[i]() > 0.5])

Coût total: 5610.0
Sites ouverts: [2, 3]


In [99]:
# Création du modèle
rmp = ConcreteModel()
rmp.sites = Set(initialize=sites)
rmp.clients = Set(initialize=clients)
rmp.y = Var(rmp.sites, within=Binary, initialize=0)
rmp.phi = Var(within=NonNegativeReals)
rmp.obj_master = Objective(
    expr=sum(fixed_cost[i] * rmp.y[i] for i in rmp.sites) +
         rmp.phi,
    sense=pyo.minimize
)
# rmp.facility_constraint = Constraint(expr=sum(rmp.y[i] for i in rmp.sites) <= K)
# rmp.facility_constraint = Constraint(expr=sum(rmp.y[i] for i in rmp.sites) >= 1)
rmp.c_phi = ConstraintList()


sp = ConcreteModel()
sp.sites = Set(initialize=sites)
sp.clients = Set(initialize=clients)
sp.alpha = Var(sp.clients, within=NonNegativeReals, initialize=0)
sp.beta = Var(sp.sites, within=NonNegativeReals, initialize=0)
sp.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
solver = SolverFactory('cbc')

i = 0
lb = 0
ub = 1e6
while (ub - lb) >= 1:
    print("*"* 10 + " " + str(i))
    # Résolution du problème esclave (Benders)
    results_master = solver.solve(rmp, tee=False)

    if (results_master.solver.status == SolverStatus.ok) and \
        (results_master.solver.termination_condition == TerminationCondition.optimal) and \
        lb < pyo.value(rmp.obj_master):
        lb = pyo.value(rmp.obj_master)
        print(f"New lower bound = {pyo.value(rmp.obj_master)}")
    
    # Résolution du problème esclave (Benders)
    sp.del_component('obj')
    sp.del_component('c1')
    sp.del_component('c1_index')
    sp.del_component("c_ray")
    sp.obj = Objective(expr=sum([demand[i] * sp.alpha[i] for i in sp.clients]) - sum([M[j] * rmp.y[j].value * sp.beta[j] for j in sp.sites]), sense = pyo.maximize)
    sp.c1 = Constraint(sp.clients, sp.sites, rule=lambda m, i, j: sp.alpha[i] - sp.beta[j] <= serve_cost[(i,j)])

    results_subproblem = solver.solve(sp, tee=False)
    #print(results_subproblem)
    if (results_subproblem.solver.status == SolverStatus.ok) and (results_subproblem.solver.termination_condition == TerminationCondition.optimal):
        tmp_ub = sum(fixed_cost[i] * rmp.y[i].value for i in rmp.sites) + pyo.value(sp.obj)
        if tmp_ub < ub:
            ub = tmp_ub
            print(f"New upper bound = {ub}")
        print("Benders optimality cuts")
        # Benders optimality cuts
        rmp.c_phi.add(expr=rmp.phi >= sum([demand[i] * sp.alpha[i].value for i in sp.clients]) - sum([M[j] * rmp.y[j] * sp.beta[j].value for j in sp.sites]))
    else:
        # Benders feasibility cuts
        sp.del_component('obj')
        sp.del_component('c1')
        sp.del_component('c1_index')
        # Calculate one extreme ray
        # To find the extreme rays, we first insert 0 on the right-hand side.
        sp.c1 = Constraint(sp.clients, sp.sites, rule=lambda m, i, j: sp.alpha[i] - sp.beta[j] <= 0)
        # We set dummy
        sp.obj = Objective(expr=0, sense = pyo.maximize)
        sp.c_ray = Constraint(expr=sum([demand[i] * sp.alpha[i] for i in sp.clients]) - sum([M[j] * rmp.y[j].value * sp.beta[j] for j in sp.sites]) == 1)

        results_subproblem = solver.solve(sp, tee=False)
        #print(results_subproblem)
        # Infeasible (unbounded), generate ray u to find the violated feasibility constraint
        # Given ∗ computed by the RMP, if (dual) subproblem is unbounded, 
        # add a cut determined by an extreme ray in the dual space to the RMP
        rmp.c_phi.add(expr=0 >= sum([demand[i] * sp.alpha[i].value for i in sp.clients]) - sum([M[j] * rmp.y[j] * sp.beta[j].value for j in sp.sites]))
        
        
    i = i + 1

# Affichage de la solution
print(f"Bounds {(lb, ub)}")
print("Coût total:", rmp.obj_master())
print("Sites ouverts:", [i for i in rmp.sites if rmp.y[i]() > 0.5])

********** 0
model.name="unknown";
    - termination condition: unbounded
    - message from solver: <undefined>
********** 1
New lower bound = 1000.0
model.name="unknown";
    - termination condition: unbounded
    - message from solver: <undefined>
********** 2
New lower bound = 2000.0
New upper bound = 6840.0
Benders optimality cuts
********** 3
New lower bound = 3840.0
New upper bound = 5610.0
Benders optimality cuts
********** 4
New lower bound = 4840.0
Benders optimality cuts
********** 5
New lower bound = 5550.0
Benders optimality cuts
********** 6
New lower bound = 5610.0
Benders optimality cuts
Bounds (5610.0, 5610.0)
Coût total: 5610.0
Sites ouverts: [2, 3]
